In [15]:
import pandas as pd
import sqlite3
import os

In [16]:
#Step 1: Load Raw Data Files (CSV + JSON)
orders = pd.read_csv("orders_unstructured_data.csv")
customers = pd.read_json("customers_messy_data.json")
products = pd.read_json("products_inconsistent_data.json")

In [17]:
#Step 2: Normalize Column Names
def normalize_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")
    return df

orders = normalize_columns(orders)
customers = normalize_columns(customers)
products = normalize_columns(products)

In [18]:
#Step 3: Preview Top Rows
print("Orders Sample:")
display(orders.head())
print("Customers Sample:")
display(customers.head())
print("Products Sample:")
display(products.head())

Orders Sample:


,order_id,ord_id,customer_id,cust_id,order_date,order_datetime,product_id,item_id,quantity,qty,...,order_total,shipping_cost,tax,discount,status,order_status,payment_method,shipping_address,notes,tracking_number
0,ORD_00001,1,316,CUST_0316,4/1/2023,2023-07-17T02:49:00.000Z,PROD_013,143,7,1,...,684.81,17.88,17.52,0.00,processing,NaN,debit_card,4648 Main St,NaN,NaN
1,ORD_00002,2,67,CUST_0067,7/28/2023,NaN,PROD_064,168,7,1,...,520.20,0.00,4.78,20.78,returned,processing,bank_transfer,346 Main St,NaN,TRK209660
2,ORD_00003,3,373,CUST_0373,7/19/2023,2023-08-14T17:27:00.000Z,PROD_115,88,7,5,...,359.42,13.51,18.81,0.00,SHIPPED,NaN,credit_card,8981 Oak Ave,NaN,TRK895514
3,ORD_00004,4,133,CUST_0133,9/1/2023,2023-09-12T11:04:00.000Z,PROD_143,110,4,4,...,236.93,17.90,40.37,0.00,pending,processing,paypal,6424 Pine Rd,NaN,TRK350215
4,ORD_00005,5,298,CUST_0298,3/3/2023,2023-09-25T19:14:00.000Z,PROD_142,114,9,2,...,568.53,15.82,13.85,11.35,delivered,CANCELLED,debit_card,8454 Oak Ave,NaN,NaN


Customers Sample:


,customer_id,cust_id,customer_name,full_name,email,email_address,phone,phone_number,address,city,...,status,customer_status,total_orders,total_spent,loyalty_points,preferred_payment,age,birth_date,gender,segment
0,1,CUST_0001,jane_doe,Bob Wilson,customer1@example.com,user1@domain.com,555-2322,(555) 818-1484,7633 Main St,Houston,...,INACTIVE,pending,17,1375.96,805,None,37.0,1990-03-04,M,vip
1,2,CUST_0002,alice.johnson@email.com,diana.prince,customer2@example.com,user2@domain.com,None,,2937 First St,Los Angeles,...,INACTIVE,suspended,21,1966.01,630,None,39.0,None,Other,premium
2,3,CUST_0003,EVE WHITE,diana.prince,customer3@example.com,user3@domain.com,None,,1641 First St,chicago,...,INACTIVE,ACTIVE,24,3421.32,96,paypal,59.0,1993-12-24,Male,regular
3,4,CUST_0004,Charlie Brown,diana.prince,customer4@example.com,user4@domain.com,None,(555) 150-1781,4153 Second Ave,NYC,...,INACTIVE,INACTIVE,42,3683.84,544,paypal,71.0,1961-07-14,F,None
4,5,CUST_0005,diana.prince,EVE WHITE,customer5@example.com,user5@domain.com,None,,1838 Second Ave,New York,...,ACTIVE,suspended,16,2865.06,190,cash,76.0,None,M,vip


Products Sample:


,product_id,item_id,product_name,item_name,description,category,product_category,brand,manufacturer,price,...,color,size,stock_quantity,stock_level,reorder_level,supplier_id,created_date,last_updated,is_active,rating
0,PROD_001,1,Product 1,Item 1 Name,Description for product 1,clothing,electronics,BRAND-C,None,314.49,...,White,,277,254,14,None,2022-12-11,2023-02-03T18:30:00.000Z,true,0.3
1,PROD_002,2,Product 2,Item 2 Name,Description for product 2,clothing,clothing,brand_b,BRAND-C,50.37,...,Black,S,938,38,23,None,2020-02-23,2023-02-23T18:30:00.000Z,false,1.9
2,PROD_003,3,Product 3,Item 3 Name,Description for product 3,Sports,Books,brandE,None,152.16,...,Green,XL,120,441,32,SUP_05,2020-04-03,2023-11-27T18:30:00.000Z,False,4.3
3,PROD_004,4,Product 4,Item 4 Name,Description for product 4,clothing,Toys,brandE,brandE,44.31,...,Green,XL,997,262,45,SUP_11,2023-12-21,2023-01-02T18:30:00.000Z,0,3.9
4,PROD_005,5,Product 5,Item 5 Name,Description for product 5,CLOTHING,Electronics,BRAND-C,BRAND-C,445.60,...,Red,XL,673,406,36,SUP_07,2023-11-04,2023-04-24T18:30:00.000Z,true,4.1


In [19]:
#Step 4: Missing Values Check
print("Missing values:")
print("Orders:\n", orders.isnull().sum())
print("Customers:\n", customers.isnull().sum())
print("Products:\n", products.isnull().sum())

Missing values:
Orders:
 order_id              0
ord_id                0
customer_id           0
cust_id               0
order_date           94
order_datetime      195
product_id            0
item_id               0
quantity              0
qty                   0
unit_price            0
price                 0
total_amount          0
order_total           0
shipping_cost         0
tax                   0
discount              0
status              146
order_status        130
payment_method        0
shipping_address      0
notes               793
tracking_number     417
dtype: int64
Customers:
 customer_id            0
cust_id                0
customer_name          0
full_name              0
email                 86
email_address          0
phone                317
phone_number           0
address                0
city                   0
state                  0
zip_code             148
postal_code            0
registration_date    100
reg_date               0
status                6

In [20]:
#Step 5: Data Types Check
print("Data types:")
print("Orders:\n", orders.dtypes)
print("Customers:\n", customers.dtypes)
print("Products:\n", products.dtypes)

Data types:
Orders:
 order_id             object
ord_id                int64
customer_id           int64
cust_id              object
order_date           object
order_datetime       object
product_id           object
item_id               int64
quantity              int64
qty                   int64
unit_price          float64
price               float64
total_amount        float64
order_total         float64
shipping_cost       float64
tax                 float64
discount            float64
status               object
order_status         object
payment_method       object
shipping_address     object
notes                object
tracking_number      object
dtype: object
Customers:
 customer_id            int64
cust_id               object
customer_name         object
full_name             object
email                 object
email_address         object
phone                 object
phone_number          object
address               object
city                  object
state              

In [21]:
#Step 6: Drop Duplicates
orders = orders.drop_duplicates()
customers = customers.drop_duplicates()
products = products.drop_duplicates()

In [22]:
#Step 7: Convert Dates
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')
if 'created_date' in products.columns:
    products['created_date'] = pd.to_datetime(products['created_date'], errors='coerce')
if 'last_updated' in products.columns:
    products['last_updated'] = pd.to_datetime(products['last_updated'], errors='coerce')

In [23]:
#Step 8: Convert Numeric Fields
for col in ['amount', 'price', 'cost', 'list_price', 'rating']:
    if col in orders.columns:
        orders[col] = pd.to_numeric(orders[col], errors='coerce')
    if col in products.columns:
        products[col] = pd.to_numeric(products[col], errors='coerce')

In [24]:
#Step 9: Standardize Booleans (e.g., is_active)
def standardize_boolean(val):
    if str(val).lower() in ['1', 'true', 'yes']:
        return True
    elif str(val).lower() in ['0', 'false', 'no']:
        return False
    return pd.NA

if 'is_active' in products.columns:
    products['is_active'] = products['is_active'].apply(standardize_boolean)

In [25]:
#Step 10: Drop rows with missing important IDs
orders = orders.dropna(subset=['order_id', 'customer_id'])
customers = customers.dropna(subset=['customer_id'])
products = products.dropna(subset=['product_id'])

In [26]:
#Step 11: Save Cleaned Files (optional)
os.makedirs("cleaned_data", exist_ok=True)
orders.to_csv("cleaned_data/orders_clean.csv", index=False)
customers.to_json("cleaned_data/customers_clean.json", orient='records', indent=2)
products.to_json("cleaned_data/products_clean.json", orient='records', indent=2)

In [27]:
#Step 12: Load into SQLite Database
conn = sqlite3.connect("ecommerce.db")
orders.to_sql("orders", conn, if_exists="replace", index=False)
customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

print("All data cleaned and loaded into 'ecommerce.db' successfully!")

All data cleaned and loaded into 'ecommerce.db' successfully!
